<a href="https://colab.research.google.com/github/shibaeff/Data-Science-Homeworks/blob/master/geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shapely folium geopandas lightgbm tqdm shap geopy pyshp

     |████████████████████████████████| 921kB 4.9MB/s 
     |████████████████████████████████| 245kB 42.3MB/s 
     |████████████████████████████████| 225kB 34.8MB/s 
     |████████████████████████████████| 11.9MB 39.3MB/s 
     |████████████████████████████████| 10.1MB 32.9MB/s 
  Created wheel for shap: filename=shap-0.30.2-cp36-cp36m-linux_x86_64.whl size=360644 sha256=38d2fe4f9c434c3f4b2de487f6fea650c88927663db3c1d5e5d53c7bc4a0132f
  Stored in directory: /root/.cache/pip/wheels/97/46/c9/b9dc708249af7eaf0951b2be62e0f6191f4de385a48e1a4cb7
  Created wheel for pyshp: filename=pyshp-2.1.0-cp36-none-any.whl size=32607 sha256=e357f04d95cede5a2a73a8f9e778a448a1a9099b8840f95257b48fd62e3fe267
  Stored in directory: /root/.cache/pip/wheels/a6/0c/de/321b5192ad416b328975a2f0385f72c64db4656501eba7cc1a
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6614 sha256=933399161ae128c73712c55ef11ccee0de53c56ad0ec94698937f1e99c63df11
  Stored in directory: /root/.cache/pip/wheels

In [0]:
import shapefile
reader = shapefile.Reader('./data/cent_fed/gis_osm_pois_free_1')

In [0]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import folium
import matplotlib as mpl
import matplotlib.cm as cm
pd.options.display.max_columns = 200

In [0]:
center = {'lat': 52.96508, 'lon': 36.07849}

In [0]:
lat_meters = 111300
lon_meters = 57476

def center_lat_to_meter(lat):
    return (lat - center['lat']) * lat_meters

def center_lon_to_meter(lon):
    return (lon - center['lon']) * lon_meters

In [0]:
max_distance  = 40000
out = []
for point in reader.iterShapeRecords():
    lon, lat = np.mean(point.shape.points, axis=0)
    lat_meter =  center_lat_to_meter(lat)
    lon_meter = center_lon_to_meter(lon)
    if np.abs(lat_meter) > max_distance or np.abs(lon_meter) > max_distance :
        continue
    out.append({'category': point.record[2], 
                'lat': lat, 
                'lon': lon,
                'lat_meter': lat_meter,
                'lon_meter': lon_meter,
               })

In [0]:
df_poi = pd.DataFrame(out)

In [0]:
df_school = df_poi[df_poi['category'] == 'school']

In [3]:
# %%bash
!mkdir ./data
!mkdir ./data/cent_fed
!wget -O ./data/central-fed.shp.zip   http://download.geofabrik.de/russia/central-fed-district-latest-free.shp.zip


--2019-10-15 13:29:32--  http://download.geofabrik.de/russia/central-fed-district-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 88.99.105.98, 88.99.142.44
Connecting to download.geofabrik.de (download.geofabrik.de)|88.99.105.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1077683652 (1.0G) [application/zip]
Saving to: ‘./data/central-fed.shp.zip’

./data/central-fed. 100%[===================>]   1.00G  19.4MB/s    in 57s     

2019-10-15 13:30:29 (18.2 MB/s) - ‘./data/central-fed.shp.zip’ saved [1077683652/1077683652]



In [5]:
!unzip ./data/central-fed.shp.zip -d ./data/cent_fed
!ls -l ./data/cent_fed/

Archive:  ./data/central-fed.shp.zip
  inflating: ./data/cent_fed/README  
 extracting: ./data/cent_fed/gis_osm_buildings_a_free_1.cpg  
  inflating: ./data/cent_fed/gis_osm_buildings_a_free_1.dbf  
  inflating: ./data/cent_fed/gis_osm_buildings_a_free_1.prj  
  inflating: ./data/cent_fed/gis_osm_buildings_a_free_1.shp  
  inflating: ./data/cent_fed/gis_osm_buildings_a_free_1.shx  
 extracting: ./data/cent_fed/gis_osm_landuse_a_free_1.cpg  
  inflating: ./data/cent_fed/gis_osm_landuse_a_free_1.dbf  
  inflating: ./data/cent_fed/gis_osm_landuse_a_free_1.prj  
  inflating: ./data/cent_fed/gis_osm_landuse_a_free_1.shp  
  inflating: ./data/cent_fed/gis_osm_landuse_a_free_1.shx  
 extracting: ./data/cent_fed/gis_osm_natural_a_free_1.cpg  
  inflating: ./data/cent_fed/gis_osm_natural_a_free_1.dbf  
  inflating: ./data/cent_fed/gis_osm_natural_a_free_1.prj  
  inflating: ./data/cent_fed/gis_osm_natural_a_free_1.shp  
  inflating: ./data/cent_fed/gis_osm_natural_a_free_1.shx  
 extracting: ./

In [14]:
df_school.head()

,category,lat,lat_meter,lon,lon_meter
69,school,52.929018,-4013.64495,36.024060,-3128.424428
85,school,52.933233,-3544.61562,36.039201,-2258.151574
97,school,53.015800,5645.10261,36.143214,3720.047886
99,school,53.016771,5753.19717,36.144358,3785.823420
106,school,53.013536,5393.09715,36.152157,4234.090240


In [15]:
df_school.loc[-1] = ["school", 52.976001, center_lat_to_meter(52.976001), 36.066626, center_lon_to_meter(36.066626)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_school.index += 1

In [17]:
df_school.sort_index()

,category,lat,lat_meter,lon,lon_meter
0,school,52.976001,1215.50730,36.066626,-681.895264
70,school,52.929018,-4013.64495,36.024060,-3128.424428
86,school,52.933233,-3544.61562,36.039201,-2258.151574
98,school,53.015800,5645.10261,36.143214,3720.047886
100,school,53.016771,5753.19717,36.144358,3785.823420
107,school,53.013536,5393.09715,36.152157,4234.090240
113,school,53.007656,4738.75332,36.137062,3366.495767
116,school,53.010409,5045.13996,36.142776,3694.930874
118,school,53.010782,5086.57695,36.145987,3879.451824
119,school,53.011542,5171.24286,36.133129,3140.425416


In [0]:
popup='<i><a href="https://первый-лицей.рф/">Лицей №1 им. М.В. Ломоносова</i>'

In [20]:
map_folium = folium.Map(location=(center['lat'], center['lon']),
                        tiles='Stamen Toner',
                        zoom_start=11)
for ind, row in df_school.iterrows():
    folium.CircleMarker([row['lat'], row['lon']],
                        radius=5,
                        fill=True, 
                        fill_opacity=0.9
                       ).add_to(map_folium)
folium.Marker([52.976001, 36.066626], popup=popup, tooltip='Pavel\'s highschool').add_to(map_folium)
map_folium